In [ ]:
!pip3 install beautifulsoup4
!pip3 install lxml
!pip3 install requests_html

In [16]:
import xml.etree.ElementTree as ET
import lxml.etree as etree
import urllib.request
import requests
import os
import datetime
from datetime import datetime as dt


In [23]:
#To download panopto links:
#################################################################### 
def download_videos(video_path,video_dict):
    #video dict = key = file title, value = url to download from 
    #check if video path is a folder, if not, make the folder
    if not os.path.isdir(video_path):
        #make directory under parent directory
        os.mkdir(video_path)
    print()
    print("Downloading Vids",end = "")
    for k in video_dict.keys():
#        print(f"{k}: {video_dict[k]}")
        print(".",end="")
        #This line actually downloads the mp4 files from the given links
        urllib.request.urlretrieve(video_dict[k],video_path+k+".mp4")
    print(f"\n\nDownloaded files located @ {video_path}")
    for k in video_dict.keys():
        print(k)
####################################################################    
def xml_to_dict(filename_path,start,end,delete):
    
    mytree = ET.parse(filename_path)
    channel = mytree.getroot()[0]
    guids = channel.findall("./item/guid")
    titles = channel.findall("./item/title")
    pubDates =  channel.findall("./item/pubDate")
    #print(child.tag, child.attrib) == tag , dict(k=attr_title,v=attr_value)\parent_path = "./"
    d = {}
    for i in range(len(guids)):
        #edit title
        title = titles[i].text.replace(" ","_")
        
        #get link
        link = guids[i].text 
        
        #get pubDateTime
        pubDate = pubDates[i].text
        pubDate_object = datetime_object(pubDate)
        
        #only add title and video url to the dictionary to download
        if start <= pubDate_object <= end:
            d[title] = link    
    if delete:
        #delete the file if the delete val is 1 
        os.remove(filename_path)
    return d
####################################################################
def rss_file_import(parent_path,url):
    #create new file
    now = dt.now()
    current_time = now.strftime("%H:%M:%S")
    filename = "rss_file_imported_" + str(current_time) + ".xml"
    file_path = parent_path+filename
    
    f = open(file_path, "a")
    f. truncate(0)
    response = requests.get(url) 
    data = response.text
    f.write(data)
    f.close()
    return parent_path+filename
####################################################################

In [24]:
# input = "DD/MM/YYY"
#input = jan,9,2020 - jan 11,2020
#Date Functions
####################################################################
def date_range_tup(user_input):
    #remove spaces
    user_input = user_input.replace(" ","")
    first = user_input.split("-")[0].split(",")
    second = user_input.split("-")[1].split(",")

    first[0] = first[0].upper()
    second[0] = second[0].upper()

    first = (first[0],int(first[1]),int(first[2]))
    second = (second[0],int(second[1]),int(second[2]))
    print(f"start date: {first}")
    print(f"end date: {second}")
    
    return date_range(first,second)
####################################################################
def date_range(start_tup,end_tup):
    sm,sd,sy = start_tup
    em,ed,ey = end_tup
    
    s_month_number = dt.strptime(sm, '%b').month
    e_month_number = dt.strptime(em, '%b').month
    
    start = dt(day=sd,month=s_month_number,year=sy)
    end = dt(day=ed,month=e_month_number,year=ey)
    #returns start and end datetime objects
    
    return (start,end)
####################################################################
def datetime_object(pubDate):
    input_list = pubDate.split(" ")
    
    d_day = int(input_list[1])
    d_month = input_list[2].upper()
    d_year = int(input_list[3])
    d_month = dt.strptime(d_month, '%b').month
    date_object = dt(day=d_day,month=d_month,year=d_year)
    return date_object
####################################################################

Download panopto videos within date range

Steps: 
1. Go to panopto on elms
2. Subscribe to RSS
3. Copy URL
4. Run script

In [25]:
# #Set new Filename
#Set this to true to put in your own inputs
#set to false if you want to be prompted
request_input = True

parent_path = "./"
if request_input:
    parent_path = input("Please enter a parent path: ")
# filename_path = path + filename

# #Set the URL
url = "your url"
if request_input:
    url = input("Please enter the url for the RSS feed: ")

# #Create the local RSS File with the filename
filename = rss_file_import(parent_path, url)
 
# #Get the date range from user:
user_date_range = "jan,9,2022 - feb,27,2022 "
if request_input:
    user_date_range = input("Enter a date range in example format (mar,9,2022 - may,10,2022): ")
start, end = date_range_tup(user_date_range)

# #Build the dictionary of videos titles:video urls to download
video_dict = xml_to_dict(parent_path+filename, start, end, 0)

# #Download all the videos in the dictionary
video_path = "./rss_videos/"
if request_input:
    video_path = input("Please enter the folder path for where the videos should be saved: ")
if video_path[-1] != "/":
    video_path += "/"
    
download_videos(video_path,video_dict)


start date: ('JAN', 9, 2022)
end date: ('FEB', 27, 2022)


Downloaded files located @ ./test_videos/
Tuesday,_January_25,_2022_at_3:25:47_PM
Thursday,_January_27,_2022_at_3:30:08_PM
Tuesday,_February_1,_2022_at_3:31:18_PM
Thursday,_February_3,_2022_at_3:29:58_PM
Tuesday,_February_8,_2022_at_3:30:24_PM
Thursday,_February_10,_2022_at_3:32:54_PM
Tuesday,_February_15,_2022_at_3:30:46_PM
Thursday,_February_17,_2022_at_3:30:58_PM
Tuesday,_February_22,_2022_at_3:32:56_PM
Thursday,_February_24,_2022_at_3:32:34_PM
